In [2]:
import pandas as pd
import numpy as np
import gc

In [12]:
df = pd.read_csv('../input/Merged_Input.csv',usecols=['Building'])

In [13]:
df.Building.value_counts()

Gallery    260068750
Name: Building, dtype: int64

In [10]:
# Only one value for feature "Building", which means its useless.
# So drop it out

In [3]:
df = pd.read_csv('../input/Merged_Input.csv',usecols=['Level'])

In [20]:
df.Level.value_counts()

Level 1     176288644
Level B1     78180446
Level5        5599660
Name: Level, dtype: int64

In [9]:
# Transform "Level" values to int

In [24]:
df.Level.unique()

array(['Level5', 'Level 1', 'Level B1'], dtype=object)

In [5]:
df_level_int8 = df.Level.map({'Level 1':1, 'Level B1':0, 'Level5':5}).astype('int8')

In [6]:
df_level_int8.memory_usage(deep=True)/(1024**3)#in GB

0.2422079723328352

In [7]:
df.Level.memory_usage(deep=True)/(1024**3)#in GB

15.568901659920812

In [4]:
# 15.6/0.24 = 64
# reduce memory usage to 1/64

In [8]:
df = pd.read_csv('../input/Merged_Input.csv',usecols=['ClientMacAddr'])

In [5]:
df.nunique()

ClientMacAddr    9586251
dtype: int64

In [7]:
# On average, each user has 27 records
df.shape[0]/9586251

27.1293491063399

In [9]:
%%time
def mapper(x):
    return int(x.replace(':',''),16)
df_mac_int64 = pd.Series()
i=0
while i<df.shape[0]:
    l = min(df.shape[0]-i,int(df.shape[0]/10))
    df_mac_int64= pd.concat([df_mac_int64,df['ClientMacAddr'].iloc[i:i+l].apply(mapper)])
    i+=l

Wall time: 3min 3s


In [10]:
df.ClientMacAddr.memory_usage(deep=True)/(1024**3)#in GB

17.92338451370597

In [11]:
df_mac_int64.memory_usage(deep=True)/(1024**3)#in GB

3.8753263652324677

In [13]:
# 17.92/3.88 = 4.6
# reduce memory cost to 1/4.6

In [17]:
df_mac_int64.memory_usage(deep=True)

4161100000

In [19]:
df=pd.DataFrame()
df['ClientMacAddr'] = df_mac_int64
df['Level']=df_level_int8

In [20]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260068750 entries, 0 to 260068749
Data columns (total 2 columns):
ClientMacAddr    int64
Level            int8
dtypes: int64(1), int8(1)
memory usage: 4.1 GB


In [ ]:
df.to_csv('../input/level_mac.csv',index=False)

In [2]:
%%time
col_dtypes = {
    "Level":'int8',
    "ClientMacAddr":'int64',
    "lat":'float32',
    "lng":'float32'
}
df = pd.read_csv('../input/Merged_Input.csv',dtype=col_dtypes,usecols=['lat','lng'])

Wall time: 1min 52s


In [3]:
df.describe()

,lat,lng
count,2.600688e+08,2.600688e+08
mean,1.290467e+00,1.038517e+02
std,3.930378e-04,2.911652e-04
min,1.289616e+00,1.038508e+02
25%,1.290171e+00,1.038515e+02
50%,1.290469e+00,1.038517e+02
75%,1.290801e+00,1.038519e+02
max,1.291139e+00,1.038522e+02


In [4]:
df.lat.nunique()

5676217

In [19]:
((df.lat - df.lat.mean()).astype('float32') - (df.lat- df.lat.mean())).abs().mean()/(df.lat- df.lat.mean()).abs().mean()

2.14956009868049e-08

In [20]:
((df.lng - df.lng.mean()).astype('float32') - (df.lng- df.lng.mean())).abs().mean()/(df.lng- df.lng.mean()).abs().mean()

1.9928068485686283e-08

In [ ]:
# changing lat and lng to "float32" has only 2e-6% loss
# it's reasonable and advantageous to take them as 'float32' 
# which we can same half of memory space (about 1GB memory per feature)

In [5]:
%%time
col_dtypes = {
    "Level":'int8',
    "ClientMacAddr":'int64',
    "lat":'float32',
    "lng":'float32'
}
df = pd.read_csv('../input/Merged_Input.csv',usecols=['localtime'])

Wall time: 2min 31s


In [90]:
%%time
df_localtime = pd.Series()
i=0
while i<df.shape[0]:
    l = min(df.shape[0]-i,int(df.shape[0]/10))
    df_localtime = pd.concat([df_localtime ,
                              pd.to_datetime(df['localtime'].iloc[i:i+l].apply(lambda x:x[:19]),
                                             format='%Y-%m-%d %H:%M:%S')])
    i+=l

Wall time: 1min 48s


In [95]:
# reduce to about 1/5
df_localtime.memory_usage(deep=True)/(1024**3)

3.8753263652324677

In [97]:
df_localtime.to_csv('../input/localtime.csv',index=False)

In [15]:
%%time
df_1 = pd.read_csv('../input/level_mac.csv',dtype={
    'Level':'int8',
    "ClientMacAddr":'int64'
})
print('Level, Id finished')
df_2 = pd.read_csv('../input/Merged_Input.csv',dtype={
    "lat":'float32',
    "lng":'float32'
} ,usecols=['lat','lng'])
print('lat, lng finished')
#df_3 =  pd.read_csv('../input/localtime.csv',names=['localtime'])
print('all finished')

Level, Id finished
lat, lng finished
all finished
Wall time: 2min 35s


In [3]:
df_1.head()

,ClientMacAddr,Level
0,62668778773016,5
1,62668778810659,5
2,62668778810659,5
3,62668778810963,5
4,62668778812803,5


In [4]:
df_2.head()

,lat,lng
0,1.290866,103.851723
1,1.290866,103.851723
2,1.290866,103.851723
3,1.290866,103.851723
4,1.290866,103.851723


In [2]:
%%time
df_3 =  pd.read_csv('../input/localtime.csv',names=['localtime'])

Wall time: 1min 41s


In [4]:
df_3.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260068750 entries, 0 to 260068749
Data columns (total 1 columns):
localtime    object
dtypes: object(1)
memory usage: 18.4 GB


In [9]:
%%time
df_localtime = pd.Series()
i=0
while i<df_3.shape[0]:
    l = min(df_3.shape[0]-i,int(df_3.shape[0]/10))
    df_localtime = pd.concat([df_localtime ,
                              pd.to_datetime(df_3['localtime'].iloc[i:i+l].apply(lambda x:x[:19]),
                                             format='%Y-%m-%d %H:%M:%S')])
    i+=l

Wall time: 1min 33s


In [13]:
print '参数列表:', str(sys.argv)
df['Level']=df_level_int8df_localtime.memory_usage(deep=True)/(1024**3)

3.8753263652324677

In [20]:
df_1[['lat','lng']] = df_2[['lat','lng']]

In [24]:
df_1['localtime'] = df_localtime

In [33]:
%%time
df_1.to_csv("../input/reduced_data.csv",index=False)

Wall time: 20min 44s


In [8]:
%%time
col_dtypes = {
    "Level":'int8',
    "ClientMacAddr":'int64',
    "lat":'float16',
    "lng":'float16'
}
df = pd.read_csv("../input/reduced_data.csv",dtype=col_dtypes,
                 parse_dates=['localtime'],infer_datetime_format=True)

Wall time: 4min 46s


In [9]:
d['Level']df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260068750 entries, 0 to 260068749
Data columns (total 5 columns):
ClientMacAddr    int64
Level            int8
lat              float16
lng              float16
localtime        datetime64[ns]
dtypes: datetime64[ns](1), float16(2), int64(1), int8(1)
memory usage: 5.1 GB


In [12]:
df.memory_usage(deep=True)

Index                    80
ClientMacAddr    2080550000
Level             260068750
lat               520137500
lng               520137500
localtime        2080550000
dtype: int64

In [26]:
df = pd.read_csv('../../input/Merged_Input.csv',usecols=['Level', 'ClientMacAddr', 'lat', 'lng', 'localtime'],chunksize=10000)

In [52]:
for d in df:
    break

In [79]:
print("Finish:"+10*'-'+' %02d '%(0),end='\r')